# Graph Algorithms in Real-Life Applications

This notebook contains ready-to-copy Python code implementing the four tasks from **Lab Assignment 3: Graph Algorithms in Real-Life Applications**.

It includes:
- Problem 1 — Social Network Friend Suggestion (BFS)
- Problem 2 — Route Finding (Bellman-Ford)
- Problem 3 — Emergency Response (Dijkstra)
- Problem 4 — Network Cable Installation (Prim's & Kruskal's)

---

## Setup & imports

In [ ]:
import time
import heapq
import matplotlib.pyplot as plt
from collections import deque, defaultdict

print('Environment ready')

## Problem 1: Social Network Friend Suggestion (BFS)

In [ ]:
def build_undirected_graph(edges):
    g = defaultdict(set)
    for u, v in edges:
        g[u].add(v)
        g[v].add(u)
    return g

def suggest_friends_bfs(graph, user, max_depth=2):
    if user not in graph:
        return []
    visited = set([user])
    q = deque([(user, 0)])
    foaf = set()
    while q:
        node, d = q.popleft()
        if d >= max_depth:
            continue
        for nbr in graph[node]:
            if nbr not in visited:
                visited.add(nbr)
                q.append((nbr, d+1))
                if d+1 == max_depth and nbr not in graph[user]:
                    foaf.add(nbr)
    return sorted(foaf)

edges = [('A','B'), ('A','C'), ('B','D'), ('C','E'), ('D','F'), ('E','F'), ('G','H')]
G = build_undirected_graph(edges)
print('Graph adjacency:', {k:sorted(list(v)) for k,v in G.items()})
print("Suggested friends for 'A':", suggest_friends_bfs(G, 'A'))

## Problem 2: Route Finding (Bellman-Ford)

In [ ]:
def bellman_ford(nodes, edges, src):
    INF = float('inf')
    dist = {n: INF for n in nodes}
    pred = {n: None for n in nodes}
    dist[src] = 0
    for _ in range(len(nodes)-1):
        updated = False
        for u, v, w in edges:
            if dist[u] != INF and dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                pred[v] = u
                updated = True
        if not updated:
            break
    negative_cycle = False
    for u, v, w in edges:
        if dist[u] != INF and dist[u] + w < dist[v]:
            negative_cycle = True
            break
    return dist, pred, negative_cycle

nodes = ['S','A','B','C','D']
edges = [('S','A',4), ('S','B',5), ('A','C',-6), ('B','A',-2), ('C','D',3), ('D','B',1)]
dist, pred, neg = bellman_ford(nodes, edges, 'S')
print('Distances:', dist)
print('Predecessors:', pred)
print('Negative cycle present?', neg)

## Problem 3: Emergency Response System (Dijkstra)

In [ ]:
def dijkstra(graph, src):
    INF = float('inf')
    dist = {n: INF for n in graph}
    pred = {n: None for n in graph}
    dist[src] = 0
    pq = [(0, src)]
    while pq:
        d, node = heapq.heappop(pq)
        if d > dist[node]:
            continue
        for nbr, w in graph[node]:
            nd = d + w
            if nd < dist[nbr]:
                dist[nbr] = nd
                pred[nbr] = node
                heapq.heappush(pq, (nd, nbr))
    return dist, pred

Gw = {'A':[('B',5),('C',2)], 'B':[('A',5),('C',1),('D',3)], 'C':[('A',2),('B',1),('D',7)], 'D':[('B',3),('C',7)]}
dist, pred = dijkstra(Gw, 'A')
print('Shortest distances:', dist)
print('Predecessors:', pred)

## Problem 4: Network Cable Installation (Prim's & Kruskal's MST)

In [ ]:
class UnionFind:
    def __init__(self, nodes):
        self.parent = {n: n for n in nodes}
        self.rank = {n: 0 for n in nodes}
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    def union(self, x, y):
        rx, ry = self.find(x), self.find(y)
        if rx == ry:
            return False
        if self.rank[rx] < self.rank[ry]:
            self.parent[rx] = ry
        else:
            self.parent[ry] = rx
            if self.rank[rx] == self.rank[ry]:
                self.rank[rx] += 1
        return True

def prim_mst(graph, start=None):
    if not graph:
        return 0, []
    if start is None:
        start = next(iter(graph))
    visited = set([start])
    pq = []
    for v, w in graph[start]:
        heapq.heappush(pq, (w, start, v))
    total, mst_edges = 0, []
    while pq and len(visited) < len(graph):
        w, u, v = heapq.heappop(pq)
        if v in visited:
            continue
        visited.add(v)
        total += w
        mst_edges.append((u, v, w))
        for nbr, wt in graph[v]:
            if nbr not in visited:
                heapq.heappush(pq, (wt, v, nbr))
    return total, mst_edges

def kruskal_mst(nodes, edge_list):
    uf = UnionFind(nodes)
    edges_sorted = sorted(edge_list, key=lambda x: x[2])
    mst = []
    total = 0
    for u, v, w in edges_sorted:
        if uf.union(u, v):
            mst.append((u, v, w))
            total += w
    return total, mst

nodes = ['A','B','C','D']
edge_list = [('A','B',1),('A','C',4),('B','C',2),('B','D',5),('C','D',3)]
G_adj = {n: [] for n in nodes}
for u,v,w in edge_list:
    G_adj[u].append((v,w))
    G_adj[v].append((u,w))
print('Prim MST:', prim_mst(G_adj, start='A'))
print('Kruskal MST:', kruskal_mst(nodes, edge_list))